In [13]:
import pandas as pd
import numpy as np

In [14]:
space_units_df = pd.read_csv('../Data/space_units_df.csv', index_col='Name')
ground_units_df= pd.read_csv('../Data/ground_units_df.csv', index_col='Name')

In [101]:
space_units_df

,Ability,Bonus_Abilities,Has_Sustain_Damage,Cost,Combat,Combat_Value,Shots,Move,Capacity,Unit Type,Faction
Name,,,,,,,,,,,
Advanced Carrier,NaN,NaN,False,3,9,9,1,1^,6^,Carrier,NaN
Advanced Carrier II,Sustain Damage,NaN,True,3,9,9,1,2,8,Carrier,NaN
Arc Secundus,Other players' units in this system lose PLANE...,"At the start of each space combat round, repai...",True,8,5 (x2),5,2,1,3,Flagship,The Barony of Letnev
Artemiris,Other players must spend 2 influence to activa...,Sustain Damage,True,8,7 (x2),7,2,1,6,Flagship,The Council Keleres
Arvicon Rex,During combat against an opponent whose comman...,Sustain Damage,True,8,5 (x2),5,2,1,3,Flagship,The Mahact Gene Sorcerers
C'Morran N'orr,Apply +1 to the result of each of your other s...,Sustain Damage,True,8,6 (x2),6,2,1,3,Flagship,Sardakk N'orr
Carrier,NaN,NaN,False,3,9,9,1,1^,4^,Carrier,NaN
Carrier II,NaN,NaN,False,3,9,9,1,2,6,Carrier,NaN
Cruiser,NaN,NaN,False,2,7^,7,1,2^,-^,Cruiser,NaN


In [16]:
# Set Units

faction_A_units = {'Infantry': 2, 'Dreadnought': 2}
faction_B_units = {'Infantry': 4}

In [17]:
# Split Units into Ships and Ground Forces

def split_units(units):
    ships = {}
    ground_forces = {}

    for unit, value in units.items():
        if unit in space_units_df.index:
            ships[unit] = value
        elif unit in ground_units_df.index:
            ground_forces[unit] = value
            
    return ships, ground_forces

faction_A_ships, faction_A_ground_forces = split_units(faction_A_units)
faction_B_ships, faction_B_ground_forces = split_units(faction_B_units)

In [18]:
# Find Combat Values

def get_unit_stats(faction_units, units_df):
    faction_unit_stats = []
    for unit in faction_units:
        number_units = faction_units[unit]
        for i in range(number_units):
            name = unit
            unit_combat_value = int(units_df.loc[unit]['Combat_Value'])
            has_sustain_damage = units_df.loc[unit]['Has_Sustain_Damage']
            shots = units_df.loc[unit]['Shots']
            unit_stats = {'Name': name, 'Unit_Combat_Value': unit_combat_value, 'Shots': shots, 'Has_Sustain_Damage': has_sustain_damage}
            faction_unit_stats.append(unit_stats)
    return faction_unit_stats

def get_ship_stats(faction_units):
    faction_unit_stats = get_unit_stats(faction_units, space_units_df)
    return faction_unit_stats

def get_ground_force_stats(faction_units):
    faction_unit_stats = get_unit_stats(faction_units, ground_units_df)
    return faction_unit_stats

faction_A_ground_forces = get_ground_force_stats(faction_A_ground_forces)
faction_B_ground_forces = get_ground_force_stats(faction_B_ground_forces)
faction_A_ships = get_ship_stats(faction_A_ships)
faction_B_ships = get_ship_stats(faction_B_ships)

In [19]:
# Determine Hits

def get_hits(faction_units):
    hits = 0
    for unit in faction_units:
        for i in range(unit['Shots']):
            roll = np.random.randint(1, 11)
            if roll >= unit['Unit_Combat_Value']:
                hits += 1
    return hits

faction_A_hits = get_hits(faction_A_ships)
faction_B_hits = get_hits(faction_B_ships)

In [20]:
# Assign Hits

def assign_hits(hits, faction_to_assign):
    extra_hits = 0
    for i in range(hits):
        try:
            if faction_to_assign[0]['Has_Sustain_Damage']:
                faction_to_assign[0]['Has_Sustain_Damage'] = False
            else:
                faction_to_assign.remove(faction_to_assign[0])
        except IndexError:
            extra_hits += 1
    return extra_hits

extra_hits_A = assign_hits(faction_A_hits, faction_B_ships)
extra_hits_B = assign_hits(faction_B_hits, faction_A_ships)

In [21]:
faction_A_ships

[{'Name': 'Dreadnought',
  'Unit_Combat_Value': 5,
  'Shots': np.int64(1),
  'Has_Sustain_Damage': np.True_},
 {'Name': 'Dreadnought',
  'Unit_Combat_Value': 5,
  'Shots': np.int64(1),
  'Has_Sustain_Damage': np.True_}]

In [22]:
faction_B_ships

[]

In [104]:
# MAIN

# For this purpose faction_A is the attacker and faction_B is the defender
# If faction_B wins there is no ground combat.


faction_A_space_wins, faction_B_space_wins, space_draws = 0, 0, 0
faction_A_ground_wins, faction_B_ground_wins, ground_draws = 0, 0, 0
space_round_counts = []
ground_round_counts = []

for i in range(0, 1000):

    faction_A_units = {'Fighter': 4, 'Cruiser': 1, 'Carrier': 1, 'Matriarch': 1, 'War Sun': 1, 'Infantry': 5}
    faction_B_units = {'Fighter': 8, 'Destroyer': 2, 'Carrier': 1, 'Dreadnought': 3, 'Destroyer': 2, 'Hil Colish': 1, 'Infantry': 5}

    faction_A_ships, faction_A_ground_forces = split_units(faction_A_units)
    faction_B_ships, faction_B_ground_forces = split_units(faction_B_units)

    # Find Combat Values
    faction_A_ships = get_ship_stats(faction_A_ships)
    faction_B_ships = get_ship_stats(faction_B_ships)
    faction_A_ground_forces = get_ground_force_stats(faction_A_ground_forces)
    faction_B_ground_forces = get_ground_force_stats(faction_B_ground_forces)

    space_round_count = 0
    while faction_A_ships and faction_B_ships:
        # Determine Hits
        faction_A_hits = get_hits(faction_A_ships)
        faction_B_hits = get_hits(faction_B_ships)

        # Assign Hits
        assign_hits(faction_A_hits, faction_B_ships)
        assign_hits(faction_B_hits, faction_A_ships)

        space_round_count += 1
    
    space_round_counts.append(space_round_count)

    if faction_A_ships:
        faction_A_space_wins += 1

        ground_round_count = 0
        while faction_A_ground_forces and faction_B_ground_forces:
            # Determine Hits
            faction_A_hits = get_hits(faction_A_ground_forces)
            faction_B_hits = get_hits(faction_B_ground_forces)

            # Assign Hits
            assign_hits(faction_A_hits, faction_B_ground_forces)
            assign_hits(faction_B_hits, faction_A_ground_forces)

            ground_round_count += 1
        
        ground_round_counts.append(space_round_count)

        if faction_A_ground_forces:
            faction_A_ground_wins += 1
        elif faction_B_ground_forces:
            faction_B_ground_wins += 1
        else:
            ground_draws += 1

    elif faction_B_ships:
        faction_B_space_wins += 1
    else:
        space_draws += 1

average_num_space_rounds = sum(space_round_counts) / len(space_round_counts)
print(f'Average Number of Space Rounds: {average_num_space_rounds:.2f}')

average_num_ground_rounds = sum(ground_round_counts) / len(ground_round_counts)
print(f'Average Number of Ground Rounds: {average_num_ground_rounds:.2f}')

total_space_games = faction_A_space_wins + faction_B_space_wins + space_draws
print(f'Number of Space Games Ran: {total_space_games}')

total_ground_games = faction_A_ground_wins + faction_B_ground_wins + ground_draws
print(f'Number of Ground Games Ran: {total_ground_games}')

df = pd.DataFrame([{'Faction A wins': faction_A_space_wins, 'Faction B wins': faction_B_space_wins, 'Draw': space_draws},
                   {'Faction A wins': faction_A_ground_wins, 'Faction B wins': faction_B_ground_wins, 'Draw': ground_draws}],
                   index=['Space Percentages', 'Ground Percentages'])

divisors = pd.Series([total_space_games, total_ground_games], index=df.index)
df = df.div(divisors, axis=0)

faction_A_wins_both = df.loc['Space Percentages', 'Faction A wins'] * df.loc['Ground Percentages', 'Faction A wins']
faction_B_wins_either = df.loc['Space Percentages', 'Faction B wins'] + (df.loc['Space Percentages', 'Faction A wins'] * df.loc['Ground Percentages', 'Faction B wins'])
draw_either = df.loc['Space Percentages', 'Draw'] + df.loc['Ground Percentages', 'Draw']
df.loc['Overall Percentages'] = {'Faction A wins': faction_A_wins_both, 'Faction B wins': faction_B_wins_either, 'Draw': draw_either}
df = df * 100

df.round(1)

Average Number of Space Rounds: 2.98
Average Number of Ground Rounds: 5.21
Number of Space Games Ran: 1000
Number of Ground Games Ran: 14


,Faction A wins,Faction B wins,Draw
Space Percentages,1.4,97.8,0.8
Ground Percentages,64.3,35.7,0.0
Overall Percentages,0.9,98.3,0.8


In [103]:
df

,Faction A wins,Faction B wins,Draw
Space Percentages,1.400000,98.600000,0.0
Ground Percentages,64.285714,35.714286,0.0
Overall Percentages,0.900000,99.100000,0.0
